In [202]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
token = spotipy.util.prompt_for_user_token('Julien Salomon',
                           client_id='3f0268c00c604e70aace4982367723c7',
                           client_secret='c9a4a4f2699642c795cb82595c2c8035',
                            redirect_uri='http://localhost:8888')
spotify = spotipy.Spotify(auth=token)
spotify.trace = False # turn on tracing
spotify.trace_out = False # turn on trace out

In [303]:
all_songs = pd.read_csv('../data/final_df.csv')
all_songs[['loudness']] =  0.496000 - all_songs[['loudness']]
features = ["acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", "loudness","mode", "speechiness", "tempo", "time_signature", "valence"]
normalized_songs = all_songs[['artist', 'name_track', 'name_album'] + features]
normalized_songs[features] = normalized_songs[features] / all_songs[features].max()
normalized_songs

/Users/jsalomon/Anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,artist,name_track,name_album,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Frank Sinatra,Nice 'N' Easy - Remastered,Sinatra Sings Alan & Marilyn Bergman,0.881288,0.497404,0.170831,0.224261,0.000000,0.000000,0.112121,0.397076,1.0,0.034063,0.525406,0.8,0.480612
1,Frank Sinatra,Ol' Macdonald,Sinatra Sings Alan & Marilyn Bergman,0.742455,0.631360,0.165185,0.414883,0.000000,1.000000,0.182828,0.311886,0.0,0.039062,0.405033,0.8,0.744898
2,Frank Sinatra,Sentimental Baby,Sinatra Sings Alan & Marilyn Bergman,0.904427,0.301142,0.161036,0.148828,0.000505,0.090909,0.170707,0.509874,1.0,0.030625,0.393315,0.6,0.290816
3,Frank Sinatra,Love Looks So Well On You,Sinatra Sings Alan & Marilyn Bergman,0.906439,0.210800,0.163729,0.292559,0.000461,0.727273,0.108081,0.364372,1.0,0.033021,0.408479,0.8,0.205102
4,Frank Sinatra,Sleep Warm,Sinatra Sings Alan & Marilyn Bergman,0.934608,0.151610,0.171120,0.177370,0.014315,0.909091,0.111111,0.521930,1.0,0.033333,0.420585,1.0,0.141837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,Eminem,Curtains Up,Encore (Deluxe Version),0.482897,0.317757,0.048292,0.738022,0.000990,0.818182,0.944444,0.642588,0.0,0.093438,0.549235,0.6,0.027449
1940,Eminem,Encore/Curtains Down,Encore (Deluxe Version),0.084306,0.628245,0.359003,0.761468,0.000000,0.454545,0.466667,0.202151,0.0,0.413542,0.831339,0.8,0.576531
1941,Eminem,Curtains Close,The Eminem Show,0.521127,0.593977,0.063749,0.419980,0.000262,1.000000,0.337374,0.690046,1.0,0.562500,0.474637,0.6,0.698980
1942,Eminem,Bitch,The Slim Shady LP (Expanded Edition),0.995976,0.866044,0.020306,0.349643,0.012399,1.000000,0.623232,0.542878,0.0,0.912500,0.464491,0.6,0.988776


In [304]:
yesterdays = all_songs[all_songs.name_track.str.contains("Yesterday - Remastered 2015")]

In [305]:
uris = [track['uri'] for track in spotify.album(spotify.search("My Way", type="album")['albums']['items'][2]['uri'])['tracks']['items']]

In [306]:
for uri in uris:
    print(spotify.track(uri)['name'])
yesterday_uri = uris[3]

Watch What Happens
Didn't We
Hallelujah, I Love Her So
Yesterday
All My Tomorrows
My Way
A Day In The Life Of A Fool
For Once In My Life
If You Go Away
Mrs. Robinson
For Once In My Life - Rehearsal
My Way - Live At The Reunion Arena/1987


In [307]:
sinatra_yesterday = spotify.audio_features(yesterday_uri)[0]
sinatra_yesterday['artist'] = 'Frank Sinatra'
sinatra_yesterday['name_track'] = 'Yesterday'
sinatra_yesterday['loudness'] = 0.496000 - sinatra_yesterday['loudness']
for feature in features:
    sinatra_yesterday[feature] /= all_songs[feature].max()

In [308]:
yesterdays = yesterdays.append(sinatra_yesterday, ignore_index=True).dropna(axis=1)

In [309]:
duo = yesterdays.append(all_songs[all_songs.name_track.str.contains("Something")].iloc[[1,5]])
duo = duo.dropna(axis=1)
duo.name_track = duo.apply(lambda row: row['name_track'].replace(" - Remastered 2015", ""), axis=1)
cols = ["artist", "name_track", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", "loudness","mode", "speechiness", "tempo", "time_signature", "valence"]

duo = duo.append(all_songs[all_songs.artist == 'The Beatles'][cols].groupby('artist').mean().reset_index(), ignore_index=True)
duo = duo.append(all_songs[all_songs.artist == 'Frank Sinatra'][cols].groupby('artist').mean().reset_index(), ignore_index=True)
duo[cols].to_csv('sinatra-beatles.csv')

In [310]:
duo[cols]

,artist,name_track,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,The Beatles,Yesterday,0.871000,0.401000,125320.000000,0.189000,0.000000,0.000000,0.096800,11.685000,1.000000,0.028700,94.509000,4.000000,0.357000
1,Frank Sinatra,Yesterday,0.584507,0.225337,0.246686,0.262997,0.000016,0.000000,0.112121,0.443468,1.000000,0.037604,0.618447,0.800000,0.067551
2,Frank Sinatra,Something,0.399000,0.141000,283493.000000,0.209000,0.000407,2.000000,0.108000,17.004000,1.000000,0.040800,64.556000,4.000000,0.063400
3,The Beatles,Something,0.302000,0.449000,181413.000000,0.375000,0.000002,0.000000,0.144000,9.825000,1.000000,0.028600,133.845000,4.000000,0.466000
4,The Beatles,NaN,0.417032,0.540214,163777.507212,0.501513,0.073185,4.641827,0.256011,11.550147,0.783654,0.106216,120.069322,3.858173,0.608452
5,Frank Sinatra,NaN,0.740215,0.410970,193973.230380,0.357773,0.023176,4.901266,0.505137,13.827729,0.774684,0.117767,111.998387,3.762025,0.425907


In [311]:
rihanna_eminem = normalized_songs[normalized_songs.name_track.str.contains("Love The Way")].iloc[[1,4]]
rihanna_eminem = rihanna_eminem.append(normalized_songs[normalized_songs.name_track.str.contains("The Monster")].iloc[0])
rihanna_eminem = rihanna_eminem.append(normalized_songs[normalized_songs.name_track.str.contains("Numb")].iloc[0])
rihanna_eminem = rihanna_eminem.append(normalized_songs[normalized_songs.artist == 'Eminem'].groupby('artist').mean().reset_index(), ignore_index=True)
rihanna_eminem = rihanna_eminem.append(normalized_songs[normalized_songs.artist == 'Rihanna'].groupby('artist').mean().reset_index(), ignore_index=True)

In [317]:
rihanna_eminem.loc[rihanna_eminem.name_track.isna(), 'name_track'] = 'mean'
rihanna_eminem[cols].to_csv('rihanna-eminem.csv')

In [318]:
normalized_songs[normalized_songs.artist == 'Michael Jackson'].groupby('artist').mean().reset_index().append(normalized_songs[normalized_songs.name_track.str.contains('Just Lose')].iloc[0]).to_csv('mj-eminem.csv')#.plot.bar(figsize=(20,20))

In [301]:
normalized_songs[normalized_songs.name_track.str.contains('Til You')].append(normalized_songs[normalized_songs.name_track.str.contains('Stop The Music')]).iloc[[0, 3]].to_csv('rihanna_mj.csv')

In [324]:
normalized_songs[(normalized_songs.name_album == 'Off the Wall') | (normalized_songs.name_album == 'Thriller') | (normalized_songs.name_album == 'Bad (Remastered)')]

,artist,name_track,name_album,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
957,Michael Jackson,Bad - 2012 Remaster,Bad (Remastered),0.004648,0.817238,0.254888,0.906218,0.426411,0.727273,0.067172,0.142605,1.0,0.041354,0.545259,0.8,0.402041
958,Michael Jackson,The Way You Make Me Feel - 2012 Remaster,Bad (Remastered),0.054728,0.910696,0.307193,0.870540,0.000055,0.090909,0.145455,0.167150,1.0,0.153125,0.547079,0.8,0.551020
959,Michael Jackson,Speed Demon - 2012 Remaster,Bad (Remastered),0.000782,0.819315,0.249392,0.762487,0.482863,0.545455,0.307071,0.182969,0.0,0.054271,0.472658,0.8,0.239796
960,Michael Jackson,Liberian Girl - 2012 Remastered Version,Bad (Remastered),0.466801,0.777778,0.239885,0.556575,0.001925,0.727273,0.127273,0.256869,0.0,0.107292,0.502920,0.8,0.463265
961,Michael Jackson,Just Good Friends - 2012 Remaster,Bad (Remastered),0.580483,0.892004,0.253624,0.952090,0.000016,1.000000,0.633333,0.119093,0.0,0.111458,0.568184,0.8,0.777551
962,Michael Jackson,Another Part of Me - 2012 Remaster,Bad (Remastered),0.110664,0.809969,0.241863,0.884811,0.000077,0.909091,0.355556,0.166384,1.0,0.073437,0.506920,0.8,0.788776
963,Michael Jackson,Man in the Mirror - 2012 Remaster,Bad (Remastered),0.523139,0.839045,0.328378,0.827727,0.000080,0.727273,0.135354,0.183435,1.0,0.039688,0.479497,0.8,0.274490
964,Michael Jackson,I Just Can't Stop Loving You (feat. Siedah Gar...,Bad (Remastered),0.671026,0.682243,0.259422,0.476045,0.000002,0.454545,0.029798,0.313751,1.0,0.035521,0.478580,0.8,0.281633
965,Michael Jackson,Dirty Diana - 2012 Remaster,Bad (Remastered),0.195171,0.772586,0.289304,0.830785,0.000265,0.272727,0.306061,0.150165,1.0,0.046458,0.627933,0.8,0.354082
966,Michael Jackson,Smooth Criminal - 2012 Remaster,Bad (Remastered),0.248491,0.885774,0.265604,1.000000,0.471774,0.727273,0.309091,0.147967,1.0,0.078229,0.564863,0.8,0.607143
